## DarkSky API pull for fire post clustered data
### Main columns needed are latitude, longitude, datetime

In [27]:
import pandas as pd
import config

from darksky import forecast
from datetime import datetime as dt
from datetime import timedelta
from datetime import date
import datetime
import numpy as np



In [28]:
RAPIDAPI_KEY  = config.darksky_api1
RAPIDAPI_KEY2  = config.darksky_api2

In [2]:
def weather_lookup(df, key=RAPIDAPI_KEY, days_before=0, days_after=0):
    data = []
    for index, row in df.iterrows():
        ts = row['datetime'].isoformat() 
        lat = row['latitude']
        lon = row['longitude']
        weather = forecast(key, lat, lon, time=ts)
        w_dict = weather['currently']
        w_dict['timestamp'] = ts
        w_dict['latitude'] = lat
        w_dict['longitude'] = lon
        data.append(w_dict)
    return data


In [3]:
filename = "fire_lat_lon_data/Emissions2006West_DBScan Clusters.csv"

In [4]:
firedata = pd.read_csv(filename)

In [5]:
firedata['datetime'] = (pd.to_datetime(firedata.year, format='%Y') + 
                       firedata['doy'].apply(np.ceil).apply(
                           lambda x: pd.Timedelta(x, unit='D')) + timedelta(hours=12))

In [6]:
firedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3942 entries, 0 to 3941
Data columns (total 24 columns):
Unnamed: 0             3942 non-null int64
id                     3942 non-null float64
year                   3942 non-null float64
doy                    3942 non-null float64
longitude              3942 non-null float64
latitude               3942 non-null float64
grid10k                3942 non-null float64
covertype              3942 non-null float64
fuelcode               3942 non-null float64
area_burned            3942 non-null float64
prefire_fuel           3942 non-null float64
consumed_fuel          3942 non-null float64
ECO2                   3942 non-null float64
ECO                    3942 non-null float64
ECH4                   3942 non-null float64
EPM2.5                 3942 non-null float64
cwd_frac               3942 non-null float64
duff_frac              3942 non-null float64
fuel_moisture_class    3942 non-null float64
burn_source            3942 non-null flo

In [9]:
firedata1000 = firedata[0:951]

In [7]:
firedata2000 = firedata[951:1051]

In [8]:
firedata3000 = firedata[1051:2051]

In [9]:
firedata4000 = firedata[2051:3051]

In [10]:
firedata5000 = firedata[3051:4051]

In [11]:
weather_df_all = pd.read_csv('fire_lat_lon_data/weather_df_master.csv')

In [12]:
#ran on 4/18/2020
#weather_df = pd.DataFrame(weather_lookup(firedata1000))
#weather_df.to_csv('weather_df1.csv')

In [13]:
#ran on 4/20/2020
#weather_df1 = pd.DataFrame(weather_lookup(firedata2000,RAPIDAPI_KEY))
#weather_df2 = pd.DataFrame(weather_lookup(firedata3000,RAPIDAPI_KEY2))

In [14]:
#ran on 4/21/2020
weather_df1 = pd.DataFrame(weather_lookup(firedata4000,RAPIDAPI_KEY))
weather_df2 = pd.DataFrame(weather_lookup(firedata5000,RAPIDAPI_KEY2))

In [16]:
frames = [weather_df_all, weather_df1, weather_df2]

In [17]:
weather_df_all = pd.concat(frames)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [18]:
weather_df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3942 entries, 0 to 890
Data columns (total 23 columns):
Unnamed: 0             2051 non-null float64
Unnamed: 0.1           951 non-null float64
apparentTemperature    3748 non-null float64
cloudCover             2134 non-null float64
dewPoint               3710 non-null float64
humidity               3707 non-null float64
icon                   2105 non-null object
latitude               3942 non-null float64
longitude              3942 non-null float64
precipAccumulation     12 non-null float64
precipIntensity        3544 non-null float64
precipProbability      3544 non-null float64
precipType             229 non-null object
pressure               1882 non-null float64
summary                2105 non-null object
temperature            3748 non-null float64
time                   3942 non-null int64
timestamp              3942 non-null object
uvIndex                2134 non-null float64
visibility             2185 non-null float64
wind

In [32]:
#weather_df_all.to_csv('fire_lat_lon_data/weather_df_master.csv')

In [22]:
weather_df_all.head(-20)

,Unnamed: 0,Unnamed: 0.1,apparentTemperature,cloudCover,dewPoint,humidity,icon,latitude,longitude,precipAccumulation,...,pressure,summary,temperature,time,timestamp,uvIndex,visibility,windBearing,windGust,windSpeed
0,0.0,0.0,89.32,0.00,66.21,0.50,NaN,29.9614,-100.3969,NaN,...,NaN,NaN,86.86,1155747600,2006-08-16T12:00:00,9.0,9.997,180.0,17.25,9.21
1,1.0,1.0,82.84,0.00,56.64,0.41,NaN,30.3707,-103.3322,NaN,...,NaN,NaN,82.84,1150995600,2006-06-22T12:00:00,10.0,9.997,164.0,11.36,4.69
2,2.0,2.0,85.95,0.00,42.89,0.22,NaN,30.4200,-103.2347,NaN,...,NaN,NaN,85.95,1150736400,2006-06-19T12:00:00,10.0,9.997,157.0,13.20,5.37
3,3.0,3.0,84.48,0.00,28.31,0.13,clear-day,30.6933,-104.3193,NaN,...,NaN,Clear,84.48,1148403600,2006-05-23T12:00:00,10.0,9.997,218.0,8.99,2.68
4,4.0,4.0,82.00,0.75,64.37,0.58,partly-cloudy-day,31.1047,-100.5903,NaN,...,1019.0,Mostly Cloudy,80.31,1159894800,2006-10-03T12:00:00,4.0,9.997,201.0,23.59,15.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,NaN,NaN,23.43,1.00,33.09,0.94,cloudy,48.9214,-112.7010,NaN,...,1020.7,Overcast,34.53,1160244000,2006-10-07T12:00:00,2.0,9.997,315.0,30.73,19.57
867,NaN,NaN,34.74,NaN,5.76,0.29,NaN,48.1096,-118.4873,NaN,...,NaN,NaN,34.74,1140897600,2006-02-25T12:00:00,NaN,NaN,64.0,10.14,2.74
868,NaN,NaN,68.39,NaN,50.62,0.53,NaN,48.5446,-115.6807,NaN,...,NaN,NaN,68.39,1155232800,2006-08-10T12:00:00,NaN,NaN,28.0,5.67,2.34
869,NaN,NaN,75.90,NaN,51.95,0.43,NaN,48.6549,-115.2932,NaN,...,NaN,NaN,75.90,1151949600,2006-07-03T12:00:00,NaN,NaN,355.0,5.46,2.66
